In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Check this site for the latest download link
# https://www.apache.org/dyn/closer.lua/spark
!wget -q https://dlcdn.apache.org/spark/spark-3.5.2/spark-3.5.2-bin-hadoop3.tgz
!tar xf spark-3.5.2-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,452 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,700 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages 

In [6]:
from pyspark.sql import SparkSession
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.feature import VectorAssembler

spark = SparkSession.builder.appName("WineClustering").getOrCreate()

file_path = '/content/wine-clustering.csv'
wine_df = spark.read.csv(file_path, header=True, inferSchema=True)


In [7]:
wine_df_without_label = wine_df.drop("label")
assembler = VectorAssembler(inputCols=wine_df_without_label.columns, outputCol="features")
wine_data = assembler.transform(wine_df).select("features")

k_values = [2, 10, 15, 20]
df_scores = {}

for k in k_values:
    kmeans = KMeans().setK(k).setSeed(1).setFeaturesCol("features")
    model = kmeans.fit(wine_data)

    predictions = model.transform(wine_data)

    evaluator = ClusteringEvaluator(featuresCol="features", metricName="silhouette")
    silhouette = evaluator.evaluate(predictions)
    df_scores[k] = silhouette
    print(f"Silhouette score for k={k}: {silhouette}")

print("Final S Scores:", df_scores)

spark.stop()

Silhouette score for k=2: 0.8213603513331723
Silhouette score for k=10: 0.6810238482437947
Silhouette score for k=15: 0.6844672306874956
Silhouette score for k=20: 0.6393344257672275
Final S Scores: {2: 0.8213603513331723, 10: 0.6810238482437947, 15: 0.6844672306874956, 20: 0.6393344257672275}
